In [131]:
# IMPORT LIBRARIES 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler

In [253]:
from dotenv import load_dotenv
load_dotenv()
UR_API_KEY = os.getenv('DB_ID')
UR_API_SECRET = os.getenv('DB_SECRET')
USERNAME = os.getenv('DB_USER')


In [254]:
# log it to spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth

token = oauth2.SpotifyClientCredentials(
    client_id=UR_API_KEY,
    client_secret=UR_API_SECRET)
sp = spotipy.Spotify(client_credentials_manager=token)


In [255]:
# user authorization on spotipy in order to push our new playlists

import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import spotipy.util as util

cid = UR_API_KEY  # Client ID;
secret = UR_API_SECRET  # Client Secret;
username =USERNAME   # got from Spotify->account settings
#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri = 'http://localhost:3000/callback'  # works on 3000,8888 as well
client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)
if token:
 sp = spotipy.Spotify(auth=token)
else:
 print("Can't get token for", username)
# results = sp.current_user_saved_tracks()


In [ ]:
# get a list with music genres
genres = sp.recommendation_genre_seeds()
atmosferika = [ "your genres here"]
# get every song from an artist
def get_artists_from_genre(genrelist):
    artists = []

    for genre in genrelist:
        results = sp.search(q='genre:' + genre, type='artist', limit=50)
        for i, t in enumerate(results['artists']['items']):
            artists.append(t['name'])

    return artists

def get_songs_from_genre(genrelist):
    artistlist = get_artists_from_genre(genrelist)
    songs = []
    songs_names = []
    songs_artists = []
    Audio_Features = []
    for artist in artistlist:
        results = sp.search(q='artist:' + artist, type='track', limit=20)
        for i, t in enumerate(results['tracks']['items']):
            songs.append(t['id'])
            songs_names.append(t['name'])
            songs_artists.append(t['artists'][0]['name'])
            Audio_Features.append(sp.audio_features(t['id'])[0])

    return {'songs': songs, 'songs_names': songs_names, 'songs_artists': songs_artists, 'Audio_Features': Audio_Features}


songslist_atmosferika = get_songs_from_genre(atmosferika)

songslist_atmosferika


In [168]:
songslist_atmosferika2 = songslist_atmosferika.copy()


In [186]:
#cleaning the missing values
songslist_atmosferika2['Audio_Features'].pop(1947)
songslist_atmosferika2['songs'].pop(1947)
songslist_atmosferika2['songs_names'].pop(1947)
songslist_atmosferika2['songs_artists'].pop(1947)
"0SkmMypdFPCHxb14kQ8Uqt"


'Itoko Toma'

In [187]:
# convert all to dataframe
# AttributeError: 'NoneType' object has no attribute 'keys'
Audio_Features_df = pd.DataFrame(songslist_atmosferika2["Audio_Features"]
             )

songs_df = pd.DataFrame(
    {'id': songslist_atmosferika2["songs"], 'song_name': songslist_atmosferika2["songs_names"], 'artist': songslist_atmosferika2["songs_artists"]})
# merge the two dataframes
songslist_atmosferika2 = pd.merge(songs_df, Audio_Features_df, on='id')




In [ ]:
    # clean the data
songslist_atmosferika2 = songslist_atmosferika2.drop(
    ['analysis_url', 'track_href', 'type', 'uri', 'id_y', 'duration_ms', 'time_signature', ], axis=1, errors='ignore')
songslist_atmosferika2.rename(columns={'id_x': 'id'}, inplace=True)
songslist_atmosferika2["fullname"] = songslist_atmosferika2["song_name"] + \
    " - " + songslist_atmosferika2["artist"]
songslist_atmosferika2.drop_duplicates(subset="fullname", keep=False, inplace=True)
songslist_atmosferika2.drop(['song_name', 'artist'], axis=1, inplace=True)
songslist_atmosferika2.set_index('id', inplace=True)
songslist_atmosferika2.info()


In [189]:
# save to csv
songslist_atmosferika2.to_csv('songs_df_dunata.csv')


In [228]:
# SCALING DATA
songslist_atmosferika2 = songslist_atmosferika2.drop(['fullname'], axis=1, errors='ignore')
scaler = MinMaxScaler()
songs_scaled = scaler.fit_transform(songslist_atmosferika2)
songs_scaled = pd.DataFrame(
    songs_scaled, columns=songslist_atmosferika2.columns, index=songslist_atmosferika2.index)



In [229]:
# remove low energy songs
songs_scaled = songs_scaled[songs_scaled['energy'] > 0.2]
# remove low danceability songs
songs_scaled = songs_scaled[songs_scaled['danceability'] > 0.3]


In [ ]:
# validation through silhouette score
from sklearn.metrics import silhouette_score
# run it for 6 to 30 clusters and plot it
silhouette_scores = []
#apply the k mean algorithm for any possible number of k to choose the best one
for i in range(10, 18):
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(songs_scaled)
    silhouette_scores.append(silhouette_score(songs_scaled, km.labels_))

plt.plot(range(10, 18), silhouette_scores)
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette score")


In [ ]:
# RUN THE KMEANS ALGORITHM MULTIPLE TIMES IN ORDER TO GET THE BEST inertia_
from sklearn.cluster import KMeans
# for i in range(30):
#     km = KMeans(n_clusters=12, random_state=i)
#     km.fit(songs_scaled)
#     print(i, km.inertia_)
# we go for random_state=0
km = KMeans(n_clusters=25, random_state=0
            )
km.fit(songs_scaled)
songs_scaled["cluster"] = km.labels_
songs_scaled["cluster"].value_counts()
songs_scaled["cluster"].value_counts().plot(kind="bar")

In [ ]:
# GaussianMixture version

# from sklearn.mixture import GaussianMixture
# from numpy import unique
# from numpy import where
# from matplotlib import pyplot

# # define the model
# gaussian_model = GaussianMixture(n_components=12)

# # train the model
# gaussian_model.fit(songs_scaled)

# # assign each data point to a cluster
# gaussian_result = gaussian_model.predict(songs_scaled)

# # get all of the unique clusters
# gaussian_clusters = unique(gaussian_result)
# for gaussian_cluster in gaussian_clusters:
#     # get data points that fall in this cluster
#     index = where(gaussian_result == gaussian_clusters)
#     # make the plot

# songs_scaled["cluster"] = gaussian_result
# # show the Gaussian Mixture plot
# songs_scaled["cluster"].value_counts()
# songs_scaled["cluster"].value_counts().plot(kind="bar")


In [259]:
# choosing the best clusters
# for i in range(25):
#     print("d "+str(i) +"    "+str(songs_scaled.loc[songs_scaled["cluster"]
#           == i].energy.mean()))
songs_scaled.loc[songs_scaled["cluster"]
                 == 17].reset_index().to_csv('cluster_energy.csv')


In [ ]:
songs_raw = pd.read_csv('cluster_energy.csv')
songs_raw
# create a playlist for the songs raw 
sp.user_playlist_create(USERNAME, "cluster_energy",
                        public=True, description=None)
playlist_id = sp.user_playlists(USERNAME)['items'][0]['id']

In [ ]:
ids = songs_raw['id'].tolist()
 # prevent a 400 error
try:
    sp.user_playlist_add_tracks(USERNAME, playlist_id, ids)
    print("cluster created")
except:
     print("error")


In [ ]:
# create a playlist for each cluster
songs_raw = songs_scaled.reset_index()
songs_raw
for i in range(12):
    ids = songs_raw[songs_raw["fullname"].isin(cluster_dict[i])]["id"].tolist()

   # maximun 100 songs per playlist
    if len(ids) > 100:
        ids = ids[:100]
        sp.user_playlist_create(USERNAME, "cluster_" + str(i), public=True, description=None)
        playlist_id = sp.user_playlists(USERNAME)["items"][i]["id"]
        
    
        # prevent a 400 error 
        try: 
            sp.user_playlist_add_tracks(USERNAME, playlist_id, ids)
            print("cluster_" + str(i) + " created")
        except:
            print("error")  
    
#find the id through the first dataframe

